In [41]:
file_path = "data\SuchtGPT_Angebot_nexus_final.pdf"

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\nicola.rudow\AppData\Local\Temp\ipykernel_12040\3943300725.py:1: SyntaxWarning: invalid escape sequence '\S'
  file_path = "data\SuchtGPT_Angebot_nexus_final.pdf"


In [42]:
import pdfplumber
import re
import os
from PyPDF2 import PdfReader, PdfWriter
import os

In [ ]:
print(text)
print(inhalt_end)

In [ ]:
#import pdfplumber
#funktioniert für einseitige inhaltsverzeichnisse

def extract_text_and_toc_end_page(file_path):
    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            # Überprüfe, ob das Inhaltsverzeichnis gefunden wurde
            if "Inhalt" in text:
                # Extrahiere den Text nach "Inhalt"
                toc_text = text.split("Inhalt")[1]
                
                # Gehe weiter, um die letzte Seite des Inhaltsverzeichnisses zu finden
                for j in range(i + 1, len(pdf.pages)):
                    next_page_text = pdf.pages[j].extract_text()
                    if "1" in next_page_text:  # Beispiel für ein Schlüsselwort, das das Ende anzeigt
                        return toc_text, j  # j ist der Seitenindex des Kapitels
                return toc_text, len(pdf.pages)  # Wenn kein Kapitel gefunden wird, gehe davon aus, dass das Inhaltsverzeichnis die letzte Seite erreicht hat

    return None, None  # Wenn kein Inhaltsverzeichnis gefunden wird
    
toc_text, inhalt_end = extract_text_and_toc_end_page(file_path)

2


Probierzeug

In [ ]:

def extract_toc_and_end_page(file_path):
    toc_text = ""
    toc_end_page = None

    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            # Überprüfe, ob das Inhaltsverzeichnis gefunden wurde
            if "Inhalt" in text:
                toc_text = text.split("Inhalt")[1]  # Extrahiere den Text nach "Inhalt"
                toc_end_page = i  # Setze die aktuelle Seite als die letzte Seite des Inhaltsverzeichnisses
            elif toc_text:  # Wenn wir bereits im Inhaltsverzeichnis sind
                # Überprüfe, ob der Text des Inhaltsverzeichnisses noch vorhanden ist
                if toc_text not in text:
                    break  # Beende die Schleife, wenn der Text nicht mehr vorhanden ist
                toc_end_page = i  # Aktualisiere die letzte Seite des Inhaltsverzeichnisses

    return toc_text, toc_end_page

toc_text, toc_end_page_index = extract_toc_and_end_page(file_path)

if toc_text:
    print(f"Inhaltsverzeichnis:\n{toc_text}")
    print(f"Das Inhaltsverzeichnis endet auf Seite: {toc_end_page_index}")  # +1 für die 1-basierte Seitenzählung
else:
    print("Kein Inhaltsverzeichnis gefunden.")

In [ ]:

def extract_toc_and_end_page(file_path):
    toc_text = ""
    toc_end_page = None
    in_toc = False

    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if "Inhalt" in text and not in_toc:
                toc_text = text.split("Inhalt")[1]  # Extrahiere den Text nach "Inhalt"
                in_toc = True  # Wir sind jetzt im Inhaltsverzeichnis
                toc_end_page = i  # Setze die aktuelle Seite als die letzte Seite des Inhaltsverzeichnisses
            elif in_toc:  # Wenn wir im Inhaltsverzeichnis sind
                # Überprüfe, ob der Text weiterhin relevant ist
                toc_text += "\n" + text  # Füge den Text zur Inhaltsverzeichnis-Variable hinzu
                toc_end_page = i  # Aktualisiere die letzte Seite des Inhaltsverzeichnisses
                # Optional: Beende die Schleife, wenn eine bestimmte Bedingung erfüllt ist
                if len(text.strip()) == 0:  # Wenn die Seite leer ist, breche ab
                    break

    return toc_text, toc_end_page

toc_text, toc_end_page_index = extract_toc_and_end_page(file_path)

if toc_text:
    print(f"Inhaltsverzeichnis:\n{toc_text}")
    print(f"Das Inhaltsverzeichnis endet auf Seite: {toc_end_page_index + 1}")  # +1 für die 1-basierte Seitenzählung
    print(f"Länge des extrahierten Textes: {len(toc_text)} Zeichen")  # Ausgabe der Länge des extrahierten Textes
else:
    print("Kein Inhaltsverzeichnis gefunden.")

In [ ]:
import pdfplumber
import re

def extract_toc_and_end_page(file_path):
    toc_text = ""
    toc_end_page = None
    in_toc = False
    matches = []  # Liste für die gefundenen Abschnitte

    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if "Inhalt" in text and not in_toc:
                toc_text = text.split("Inhalt")[1]  # Extrahiere den Text nach "Inhalt"
                in_toc = True  # Wir sind jetzt im Inhaltsverzeichnis
                toc_end_page = i  # Setze die aktuelle Seite als die letzte Seite des Inhaltsverzeichnisses
                # Suche nach Abschnitten, die typischerweise im Inhaltsverzeichnis vorkommen
                matches = re.findall(r'(\d+\.\s.*?)(?=\n\d+\.\s|$)', toc_text, re.DOTALL)  # Beispiel für Kapitelnummern
            elif in_toc:  # Wenn wir im Inhaltsverzeichnis sind
                toc_text += "\n" + text  # Füge den Text zur Inhaltsverzeichnis-Variable hinzu
                toc_end_page = i  # Aktualisiere die letzte Seite des Inhaltsverzeichnisses

                # Überprüfen, ob wir das Ende des Inhaltsverzeichnisses erreicht haben
                if len(matches) > 0:
                    for j, match in enumerate(matches):
                        start = toc_text.find(match)  # Startposition des aktuellen Abschnitts
                        # Definiere die Endposition als den Start des nächsten Abschnitts oder das Ende des Textes
                        end = toc_text.find(matches[j + 1]) if j + 1 < len(matches) else len(toc_text)
                        section_text = toc_text[start:end].strip()  # Extrahiere den Abschnitt
                        # Optional: Hier kannst du die extrahierten Abschnitte speichern oder verarbeiten
                else:
                    break  # Beende die Schleife, wenn keine Abschnitte mehr gefunden werden

    return toc_text, toc_end_page


toc_text, toc_end_page_index = extract_toc_and_end_page(file_path)

if toc_text:
    print(f"Inhaltsverzeichnis:\n{toc_text}")
    print(f"Das Inhaltsverzeichnis endet auf Seite: {toc_end_page_index + 1}")  # +1 für die 1-basierte Seitenzählung
else:
    print("Kein Inhaltsverzeichnis gefunden.")

ende probierzeug

In [ ]:
#print(text) #entkommentieren, falls man den output testen will

In [44]:
# Regex für den ersten Abschnitt
first_pattern = r"^(?:(1(?:\.\d+)*\.?|AP 1(?:\.\d+)*\.?|AP 0\.?)(?:[.:]?)\s+([^\.\n]+))"


#Extrahieren der Abschnittsnamen im Inhaltsverzeichnis
# Regex für den ersten Abschnitt
first_pattern = r"^(?:(1(?:\.\d+)*\.?|AP 1(?:\.\d+)*\.?|AP 0\.?)(?:[.:]?)\s+([^\.\n]+))"

# Regex für alle weiteren Abschnitte
subsequent_pattern = r"^(?:(\d+\.\d*|AP \d+\.\d*|AP \d+:[.:]?|\d+)[.:]?)\s+([^\.\n]+)"

# Suche nach dem ersten Abschnitt
first_match = re.search(first_pattern, text, re.MULTILINE)

# Wenn der erste Match gefunden wurde, verarbeite die restlichen
if first_match:
    abschnitte = [f"{first_match.group(1).strip()} {first_match.group(2).strip()}"]
    
    # Schneide den Text ab dem ersten Match ab und suche nach den weiteren Abschnitten
    remaining_text = text[first_match.end():]
    subsequent_matches = re.findall(subsequent_pattern, remaining_text, re.MULTILINE)
    
    # Füge die restlichen Abschnitte hinzu
    abschnitte.extend([f"{num.strip()} {name.strip()}" for num, name in subsequent_matches])
else:
    abschnitte = []  # Falls kein gültiger erster Abschnitt gefunden wird

In [47]:
#print(abschnitte) #entkommentieren falls man testen will was gefunden wurde

['1 Projekthintergrund und Zielsetzung 4', '1.1 Projekthintergrund 4', '1.2 Die doppelte Zielsetzung des Projekts 7', '2 Konzeption, Planung und Organisation des Dialogprozesses (AP 1) 10', '2.1 AP 1 Erarbeitung des Feinkonzepts 10', '2.2 Organisation und Prozesssteuerung 12', '2.3 Qualitätssicherung und Datenschutz 13', '3 Rekrutierung von Zufallsbürger*innen sowie Eingangs- und Ausgangsbefragungen 14', '3.1 Zufallsauswahl der Bürger*innen (AP 2) 14', '3.2 Eingangsbefragung (AP 3) 15', '3.3 Ausgangsbefragung und qualitative Erhebung (AP 5) 18', '4 Durchführung der Werkstätten und Workshops 22', '4.1 Durchführung der Bürgerwerkstätten (AP 4) 22', '4.2 Durchführung des Workshops der Bürgerbotschafter*innen (AP 6) 24', '4.3 Übergabe der Empfehlungen (AP 7) 25', '5 Berichterstattung 27', '5.1 Abschlussbericht (AP 8) 27', '6 Zeitplan 28', '7 Personal und Eignung 30', '7.1 Qualifikation der Projektverantwortlichen 30', '7.2 Referenzen 35', '7.3 Institutsbeschreibungen 46', '8 Kosten- und Za

In [48]:
###PDF Aufräumen und Abschnitte extrahieren###

#Funktion um alles bis nach dem Innhaltsverzeichnis aus der pdf datei zu löschen, damit Inhaltsverzeichnis nicht erneut extrahiert wird
def remove_pages_before_contents(pdf_path, output_path, contents_start_page=inhalt_end):
    """
    Remove pages before the content starts (after the table of contents).
    
    :param pdf_path: Path to the original PDF file.
    :param output_path: Path to save the new PDF file without the first pages.
    :param contents_start_page: The page number (0-indexed) where the content starts.
    :return: None
    """
    reader = PdfReader(pdf_path)
    writer = PdfWriter()

    # Add pages starting from the contents_start_page to the new PDF
    for page_num in range(contents_start_page, len(reader.pages)):
        writer.add_page(reader.pages[page_num])

    # Write the new PDF to the output path
    with open(output_path, "wb") as output_pdf:
        writer.write(output_pdf)

    print(f"New PDF saved to: {output_path}")

#Funktion um die einzelnen Abschnitte zu finden und zu extrahieren
def extract_sections_from_pdf(pdf_path, sections):
    """
    Extract specified sections from a PDF file, starting at the match's start.

    :param pdf_path: Path to the PDF file.
    :param sections: List of section headings to extract.
    :return: Dictionary with section headings as keys and extracted text as values.
    """
    # Read the PDF
    reader = PdfReader(pdf_path)
    text = ""

    # Combine text from all pages
    for page in reader.pages:
        text += page.extract_text()

    # Prepare a regex pattern to match sections
    sections_pattern = [re.escape(sec).replace(r'\ ', r'\s+') for sec in sections]
    regex = re.compile(f"({'|'.join(sections_pattern)})", re.MULTILINE)


    # Find matches and extract content
    matches = list(regex.finditer(text))
    extracted_sections = {}


    # Loop through the matches to extract text for each section
    for i, match in enumerate(matches):
        start = match.start()  # Start from the beginning of the match

        # Define the end position as the start of the next section or the end of the text
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        # Extract the text for this section
        section_text = text[start:end].strip()

        # Store the extracted section text
        extracted_sections[match.group()] = section_text

    return extracted_sections

pdf_path = file_path
new_pdf_path = "data/cleaned_pdf.pdf"  # New PDF after removing pages up to the contents
sections = abschnitte  # Liste der Abschnitte, die du extrahieren möchtest

In [49]:
# Call Funktion to remove pages before the table of contents
remove_pages_before_contents(pdf_path, new_pdf_path, contents_start_page=2)  # Page 1 is the second page - Aktuell noch manuell die passende Seite eingegeben

New PDF saved to: data/cleaned_pdf.pdf


In [ ]:
# Extract sections from the cleaned PDF
extracted_sections = extract_sections_from_pdf(new_pdf_path, abschnitte)

In [ ]:
#Funktion um die extrahierten abschnitte in textdateien zu speichern
def save_sections_as_text_files(extracted_sections, output_directory):
    """
    Saves each section as a separate text file in the specified directory.
    
    :param extracted_sections: Dictionary with section names as keys and content as values.
    :param output_directory: Directory where the text files will be saved.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for section, content in extracted_sections.items():
        # Sanitize the section name to make it a valid filename
        filename = re.sub(r'[\\/*?:"<>|]', "_", section)  # Replace invalid characters with underscores
        file_path = os.path.join(output_directory, f"{filename}.txt")

        # Write the content into the file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)

        print(f"Section '{section}' saved to '{file_path}'")


output_directory = "extracted_sections"  # Directory to save the text files

# Funktions aufrufen, um die Abschnitte in Textdateien zu speichern
save_sections_as_text_files(extracted_sections, output_directory)